# Train DECiSION
Train a U-Net model.

## Set seeds and import packages
Settings the seeds first is meant to achieve reproducibility, though this goal is not achieved.

In [1]:
# Set Python, NumPy and TensorFlow seeds
RANDOM_STATE = 42
from numpy.random import seed
seed(RANDOM_STATE)

from tensorflow import set_random_seed
set_random_seed(RANDOM_STATE)

import random
random.seed = RANDOM_STATE

# Model and training settings
import DECiSION_settings as settings

# Toolkit imports
from dltoolkit.utils.generic import model_architecture_to_file, model_summary_to_file, list_images
from dltoolkit.nn.segment import UNet_NN
from dltoolkit.utils.visual import plot_training_history, plot_roc_curve, plot_precision_recall_curve,\
    print_confusion_matrix, print_classification_report
from dltoolkit.iomisc import HDF5Generator_Segment

from thesis_common import convert_img_to_pred, convert_pred_to_img, create_hdf5_db,\
    show_image, read_images, read_groundtruths, print_training_info
from thesis_metric_loss import dice_coef, weighted_pixelwise_crossentropy_loss

# Keras imports
from keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger
from keras.optimizers import Adam

# scikit-learn imports
from sklearn.model_selection import train_test_split

# Other imports
import numpy as np
import os, cv2, time, progressbar
import matplotlib.pyplot as plt
%matplotlib inline

/Users/geoff/anaconda3/envs/ML3-DL-OPENCV/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Change how TensorFlow allocates GPU memory
This has no effect on non-GPU machines. Setting `gpu_options.allow_growth` to `True` means TensorFlow will allocate GPU memory as needed rather than using all memory from the start. This enables monitoring of actual memory usage and determining how close the notebook gets to running out of memory.

In [2]:
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
 
# Only allow a percentage of the GPU memory to be allocated
# config.gpu_options.per_process_gpu_memory_fraction = 0.5
 
# Create a session with the above options specified
k.tensorflow_backend.set_session(tf.Session(config=config))

## Determine how to train
The variables below determine how the model will be trained:

- `USE_GENERATORS`: when set to `True` the training process will use data generators, which is typically what will be required given the size of the models being trained and the size of the images. Set to `False` to (try) to load all data into memory.
- `USE_VALIDATION_SET`: set to `True` to use a validation set during training, which will be used most of the tie. set to `False` to not use a validation set, which is used during pipeline validation.
- `USE_KFOLD_CV`: set to `True` to use k-fold cross validation during training, `False` otherwise. k-Fold cross validation requires all data to be loaded into memory and does not use a validation set, so the `USE_GENERATORS` and `USE_VALIDATION_SET` variables are both set to `False`.

In [3]:
USE_KFOLD_CV = False

USE_GENERATORS = True
USE_VALIDATION_SET = True

if USE_KFOLD_CV:
    # For now k-Fold CV only works with all data in memory
    USE_GENERATORS = False
    USE_VALIDATION_SET = False

## Convert individual images set to HDF5 data sets
This function converts the individual JPG files to HDF5 data sets. It creates up to four HDF5 files depending on whether a portion of the files is set aside to create a validation set using `settings.TRN_TRAIN_VAL_SPLIT`.

This conversion only has to be performed once rather than prior to every training run. However, it takes very little time to run. Note: if any changes are made to image pre-processing applied to MRI images and/or their ground truths, the conversion MUST be executed prior to any training. Otherwise training will occur withoug the pre-processing changes.

In [4]:
def perform_hdf5_conversion(settings):
    """Convert the individual .jpg files (images and ground truths) to four .hdf5 files:
    - Two with the training set (train_images.h5 and train_groundtruths.h5)
    - Two with the validation set (val_images.h5 and val_groundtruths.h5)
    
    If TRN_TRAIN_VAL_SPLIT is set to zero the validation set data sets are NOT generated.
    """
    # Prepare the path to the training images and ground truths
    img_exts = ".jpg"
    img_path = os.path.join(settings.TRAINING_PATH, settings.FLDR_IMAGES)
    msk_path = os.path.join(settings.TRAINING_PATH, settings.FLDR_GROUND_TRUTH)

    # Create a list of paths to the individual patient folders
    patient_fld_imgs = sorted([os.path.join(img_path, e.name) for e in os.scandir(img_path) if e.is_dir()])
    patient_fld_masks = sorted([os.path.join(msk_path, e.name) for e in os.scandir(msk_path) if e.is_dir()])

    # Obtain a list of paths to the training images and ground truths for each patient
    img_list = []
    msk_list = []
    for patient_ix, (p_fld_imgs, p_fld_masks) in enumerate(zip(patient_fld_imgs, patient_fld_masks)):
        img_list.extend(sorted(list(list_images(basePath=p_fld_imgs,
                                                validExts=img_exts)))
                        [settings.SLICE_START:settings.SLICE_END])
        msk_list.extend(sorted(list(list_images(basePath=p_fld_masks,
                                                validExts=img_exts)))
                        [settings.SLICE_START:settings.SLICE_END])

    assert(len(img_list) == len(msk_list))

    # Split the training set into a training and validation set. Consecutive slices no longer
    # belong to the same patient
    train_img, val_img, train_msk, val_msk = train_test_split(img_list, msk_list,
                                                              test_size=settings.TRN_TRAIN_VAL_SPLIT,
                                                              random_state=settings.RANDOM_STATE,
                                                              shuffle=True)
    
    # Create the HDF5 data sets
    output_paths = []

    # Training images
    output_paths.append(create_hdf5_db(train_img, "train", img_path,
                                       (settings.IMG_HEIGHT, settings.IMG_WIDTH, settings.IMG_CHANNELS),
                                       key=settings.HDF5_KEY, ext=settings.HDF5_EXT, settings=settings))

    # Training ground truths
    output_paths.append(create_hdf5_db(train_msk, "train", msk_path,
                                       (settings.IMG_HEIGHT, settings.IMG_WIDTH, settings.IMG_CHANNELS),
                                       key=settings.HDF5_KEY, ext=settings.HDF5_EXT, settings=settings,
                                       is_mask=True))

    # Validation images
    output_paths.append(create_hdf5_db(val_img, "val", img_path,
                                       (settings.IMG_HEIGHT, settings.IMG_WIDTH, settings.IMG_CHANNELS),
                                       key=settings.HDF5_KEY, ext=settings.HDF5_EXT, settings=settings))

    # Validation ground truths
    output_paths.append(create_hdf5_db(val_msk, "val", msk_path,
                                       (settings.IMG_HEIGHT, settings.IMG_WIDTH, settings.IMG_CHANNELS),
                                       key=settings.HDF5_KEY, ext=settings.HDF5_EXT, settings=settings,
                                       is_mask=True))

    return output_paths

In [5]:
if settings.IS_DEVELOPMENT:
    print("\n--- Converting images to HDF5, full paths:\n")
    hdf5_paths = perform_hdf5_conversion(settings)
    
    if settings.TRN_TRAIN_VAL_SPLIT == 0:
        print("\nValidation set NOT created.")
    else:
        print("\nCreated a {} training/validation set".format(settings.TRN_TRAIN_VAL_SPLIT))
else:
    # During development avoid performing HDF5 conversion for every run
    hdf5_paths = ["../data/MSC8002/training/train_images.h5",
                  "../data/MSC8002/training/train_groundtruths.h5",
                  "../data/MSC8002/training/val_images.h5",
                  "../data/MSC8002/training/val_groundtruths.h5"
                  ]

Creating HDF5 database 100% |###################################| Time: 0:00:00
Creating HDF5 database 100% |###################################| Time: 0:00:00
Creating HDF5 database 100% |###################################| Time: 0:00:00
Creating HDF5 database 100% |###################################| Time: 0:00:00



--- Converting images to HDF5, full paths:

../data/MSC8002/training/train_images.h5
../data/MSC8002/training/train_groundtruths.h5
../data/MSC8002/training/val_images.h5
../data/MSC8002/training/val_groundtruths.h5

Created a 0.1 training/validation set


## Set the class distribution
Assigning a higher weight to the positive class (i.e. blood vessels) means the model will pay `more attention` to that class. This is useful in the current scenario because the number of background (i.e. non-blood vessel) pixels far exceed the number of blood vessel pixels. Without setting a different class weight for the blood vessel class the model would simply assign the background class to all pixels to achieve a low loss.

In [ ]:
class_weights = [settings.CLASS_WEIGHT_BACKGROUND, settings.CLASS_WEIGHT_BLOODVESSEL]
print("Class distribution: {}".format(class_weights))

Class distribution: [1.0, 8.0]


## Create the U-Net model
Instantiate the U-Net model. Use different versions of the `build_model_XXX()` function to try different variations. Warning: Changing the model will change the name of the file the trained model will be saved to. Make sure to update the `DECiSION_test` notebook accordingly.

In [ ]:
unet = UNet_NN(img_height=settings.IMG_HEIGHT,
               img_width=settings.IMG_WIDTH,
               img_channels=settings.IMG_CHANNELS,
               num_classes=settings.NUM_CLASSES,
               dropout_rate=settings.TRN_DROPOUT_RATE)
model = unet.build_model_BRAIN_3layer(use_bn=True, use_dropout=False)
# model = unet.build_model_BRAIN_4layer(use_bn=True, use_dropout=False)

## Create paths
This cell just creates a few paths used later to save training output (e.g. the model architecture, training results and so on).

In [ ]:
# Prepare some path strings
model_path = os.path.join(settings.MODEL_PATH, "DECiSION_" + unet.title + "_ep{}.model".format(settings.TRN_NUM_EPOCH))
summ_path = os.path.join(settings.OUTPUT_PATH, "DECiSION_" + unet.title + "_model_summary.txt")
csv_path = os.path.join(settings.OUTPUT_PATH, "DECiSION_" + unet.title + "_training_ep{}_bs{}.csv".format(settings.TRN_NUM_EPOCH,
                                                                                            settings.TRN_BATCH_SIZE))

## Save/print model architecture information
Save the U-Net model's architecture to a file and print it in the cell below. Also saves a diagram of the architecture to disk.

In [ ]:
model.summary()
model_summary_to_file(model, summ_path)
model_architecture_to_file(unet.model, settings.OUTPUT_PATH + "DECiSION_" + unet.title)

## Compile the model
Set the loss function, optimiser and metric and compile the model, which is now ready for training. 

In [ ]:
# Set the optimiser, loss function and metrics
opt = Adam()
# opt = Adadelta()
metrics = [dice_coef]
loss = weighted_pixelwise_crossentropy_loss(class_weights)

# Compile
model.compile(optimizer=opt, loss=loss, metrics=metrics)

## Load data
The two cells below load the training data. Depending on the value of `USE_GENERATORS` data is either loaded using a generator or it is loaded into memory. The latter option is useful during development, the latter option is almost always required when training with a large(r) number of training images. 

### Option 1: Use generators
Nothing is actually loaded at this point. Only the generators are prepared for use with `fit_generator()`.

In [ ]:
# Prepare generators, only when NOT using cross validation
if USE_GENERATORS:
    print("Using data generators")
    # Training set generator using optional on-the-fly data augmentation
    rdr_train = HDF5Generator_Segment(hdf5_paths[0], hdf5_paths[1],
                                      batch_size=settings.TRN_BATCH_SIZE,
                                      num_classes=settings.NUM_CLASSES,
                                      converter=convert_img_to_pred, 
                                      feat_key=settings.HDF5_KEY)
    gen_train = rdr_train.generator(num_epochs=settings.TRN_NUM_EPOCH)

    # Validation set generator (never uses data augmentation)
    rdr_val = HDF5Generator_Segment(hdf5_paths[2], hdf5_paths[3],
                                    batch_size=settings.TRN_BATCH_SIZE,
                                    num_classes=settings.NUM_CLASSES,
                                    converter=convert_img_to_pred, 
                                    feat_key=settings.HDF5_KEY)
    gen_val = rdr_val.generator(num_epochs=settings.TRN_NUM_EPOCH)

    print("  Number of training samples: {}".format(rdr_train.num_images()))
    print("Number of validation samples: {}".format(rdr_val.num_images()))

    print(" steps_per_epoch: {}".format(rdr_train.num_images()/settings.TRN_BATCH_SIZE))
    print("validation_steps: {}".format(rdr_val.num_images()/settings.TRN_BATCH_SIZE))
else:
    print("Not creating generators.")

### Option 2: Load all data into memory
All data is loaded into memory.

In [ ]:
if not USE_GENERATORS:
    print("Loading all data into memory")

    train_imgs = read_images(hdf5_paths[0], settings.HDF5_KEY)
    train_grndtr = read_groundtruths(hdf5_paths[1], settings.HDF5_KEY)
    train_grndtr_ext_conv = convert_img_to_pred(train_grndtr, settings.NUM_CLASSES, settings.VERBOSE)  # softmax: 4D
    print("  Number of training samples: {}\n".format(len(train_imgs)))

    # Read the validation set (only when NOT using cross-validation)
    if not USE_KFOLD_CV:
        val_imgs = read_images(hdf5_paths[2], settings.HDF5_KEY)
        val_grndtr = read_groundtruths(hdf5_paths[3], settings.HDF5_KEY)
        val_grndtr_ext_conv = convert_img_to_pred(val_grndtr, settings.NUM_CLASSES, settings.VERBOSE)  # softmax: 4D
        print("Number of validation samples: {}\n".format(len(val_imgs)))
    else:
        print("Using cross validation, not loading a validation set")

## Train the model using generators
The two cells below perform actual training using generators. The first one uses a validation set, the second one does not.

### Train WITH a validation set

In [ ]:
if USE_GENERATORS and USE_VALIDATION_SET:
    # Prepare callbacks
    callbacks = [ModelCheckpoint(model_path, monitor="val_loss", mode="min", save_best_only=True, verbose=1),
                 EarlyStopping(monitor='val_loss',
                               min_delta=0,
                               patience=settings.TRN_EARLY_PATIENCE,
                               verbose=0,
                               mode="auto"),
                 CSVLogger(csv_path, append=False),
                 ]

    print("Training with a validation set, using generators.")
    print_training_info(unet, model_path, rdr_train.img_shape, settings, class_weights, opt, loss)
    print("Training start:\n")

    # Fit the model using generators and a validation set
    start_time = time.time()
    hist = model.fit_generator(gen_train,
#                      epochs=settings.TRN_NUM_EPOCH,
                     epochs=2,
                     steps_per_epoch=rdr_train.num_images()/settings.TRN_BATCH_SIZE,
                     verbose=2,
                     validation_data=gen_val,
                     validation_steps=rdr_val.num_images()/settings.TRN_BATCH_SIZE,
                     shuffle=True,
                     callbacks=callbacks)

    print("\n\nElapsed training time: {} min".format(int((time.time() - start_time))/60))

### Train WITHOUT a validation set

In [ ]:
if USE_GENERATORS and not USE_VALIDATION_SET:
    # Prepare callbacks
    callbacks = [
        ModelCheckpoint(model_path, monitor="loss", mode="min", save_best_only=True, verbose=1),
        EarlyStopping(monitor='loss', min_delta=0, patience=settings.TRN_EARLY_PATIENCE, verbose=0, mode="auto"),
        CSVLogger(csv_path, append=False),
        ]

    print("Training without a validation set, using generators.")
    print_training_info(unet, model_path, rdr_train.img_shape, settings, class_weights, opt, loss)
    print("Training start:\n")

    # Fit the model using a training set only
    start_time = time.time()
    hist = model.fit_generator(gen_train,
                     epochs=settings.TRN_NUM_EPOCH,
    #                  epochs=3,
                     steps_per_epoch=rdr_train.num_images()/settings.TRN_BATCH_SIZE,
                     verbose=2,
                     shuffle=True,
                     callbacks=callbacks)

    print("\n\nElapsed training time: {} min".format(int((time.time() - start_time))/60))

## Train the model with all data loaded in memory
The two cells below perform actual training using data loaded into memory. The first one uses a validation set, the second one does not.

### Train WITH a validation set

In [ ]:
if not USE_GENERATORS and USE_VALIDATION_SET:
    print("Training with a validation set, using generators.")
    print_training_info(unet, model_path, train_imgs.shape, settings, class_weights, opt, loss)
    print("Training start:\n")

    # Prepare callbacks
    callbacks = [
        ModelCheckpoint(model_path, monitor="val_loss", mode="min", save_best_only=True, verbose=1),
        EarlyStopping(monitor='val_loss', min_delta=0, patience=settings.TRN_EARLY_PATIENCE, verbose=0, mode="auto"),
        CSVLogger(csv_path, append=False),
        ]

    # Fit the model using a training set only
    start_time = time.time()
    hist = model.fit(train_imgs, train_grndtr_ext_conv,
                     epochs=settings.TRN_NUM_EPOCH,
                     batch_size=settings.TRN_BATCH_SIZE,
                     verbose=1,
                     shuffle=True,
                     validation_data=(val_imgs, val_grndtr_ext_conv),
                     callbacks=callbacks)

    print("\n\nElapsed training time: {} min".format(int((time.time() - start_time))/60))

### Train WITHOUT a validation set

In [ ]:
if not USE_GENERATORS and not USE_VALIDATION_SET:
    print("Training without a validation set, all data in memory.")
    print_training_info(unet, model_path, train_imgs.shape, settings, class_weights, opt, loss)
    print("Training start:\n")

    # Prepare callbacks
    callbacks = [
        ModelCheckpoint(model_path, monitor="loss", mode="min", save_best_only=True, verbose=1),
        EarlyStopping(monitor='loss', min_delta=0, patience=settings.TRN_EARLY_PATIENCE, verbose=0, mode="auto"),
        CSVLogger(csv_path, append=False),
        ]

    # Fit the model using a training set only
    start_time = time.time()
    hist = model.fit(train_imgs, train_grndtr_ext_conv,
                     epochs=settings.TRN_NUM_EPOCH,
                     batch_size=settings.TRN_BATCH_SIZE,
                     verbose=1,
                     shuffle=True,
                     callbacks=callbacks)

    print("\n\nElapsed training time: {} min".format(int((time.time() - start_time))/60))

## Use k-Fold Cross Validation

https://machinelearningmastery.com/evaluate-performance-deep-learning-models-keras/

https://github.com/keras-team/keras/issues/4530

https://stackoverflow.com/questions/40854232/keras-scikit-learn-using-fit-generator-with-cross-validation/40866543#40866543

https://stackoverflow.com/questions/41214527/k-fold-cross-validation-using-keras

https://www.kaggle.com/stefanie04736/simple-keras-model-with-k-fold-cross-validation

NICE: https://www.kaggle.com/zfturbo/fishy-keras-lb-1-25267/code

In [ ]:
if USE_KFOLD_CV:
    from sklearn.model_selection import KFold

    print("Training using cross validation")
    print_training_info(unet, model_path, train_imgs.shape, settings, class_weights, opt, loss)
    print("Training start:\n")

    # Set the optimiser, loss function and metrics
    opt = Adam()
    metrics = [dice_coef]
    loss = weighted_pixelwise_crossentropy_loss(class_weights)

    # Create the folds
    kfold = KFold(n_splits=settings.TRN_NUM_KFOLD_SPLITS, shuffle=True, random_state=settings.RANDOM_STATE)
    cvscores = []
    cvlosses = []
    fold_id = 1
    for train_idx, test_idx in kfold.split(train_imgs, train_grndtr_ext_conv):
        print("\n-----> Fold {} of {} <-----".format(fold_id, settings.TRN_NUM_KFOLD_SPLITS))
        print("Training: {} slices, test: {} slices".format(len(train_idx), len(test_idx)))
        
        # Create and compile the model before each fold (to reset the weights)
        model = unet.build_model_softmax()
        model.compile(optimizer=opt, loss=loss, metrics=metrics)

        # Include the number of the fold in the file name
        kfold_path = os.path.join(settings.MODEL_PATH, "DECiSION_" + unet.title + "_ep{}_kfold{}.model".format(settings.TRN_NUM_EPOCH, fold_id))

        callbacks = [
            ModelCheckpoint(kfold_path, monitor="loss", mode="min", save_best_only=True, verbose=1),
            EarlyStopping(monitor='loss', min_delta=0, patience=settings.TRN_EARLY_PATIENCE, verbose=0, mode="auto"),
            CSVLogger(csv_path, append=False),
        ]

        hist = model.fit(train_imgs[train_idx], train_grndtr_ext_conv[train_idx],
                         epochs=settings.TRN_NUM_EPOCH,
                         batch_size=settings.TRN_BATCH_SIZE,
                         verbose=2,
                         shuffle=True,
                         callbacks=callbacks)

        scores = model.evaluate(train_imgs[test_idx], train_grndtr_ext_conv[test_idx], verbose=2)
        print("\n-----> Fold {}: {}: {:.2f}%% - {}: {:.2f}\n".format(fold_id,
                                                          model.metrics_names[1],
                                                          scores[1]*100,
                                                          model.metrics_names[0],
                                                          scores[0]))
        cvscores.append(scores[1] * 100)
        cvlosses.append(scores[0])

        fold_id += 1

    print("Results")
    print("------------------")
    for i in range(NUM_SPLITS):
        print("Fold {}: {}: {:.2f}% - {}: {:.2f}\n".format(i,
                                                          model.metrics_names[1],
                                                          cvscores[i],
                                                          model.metrics_names[0],
                                                          cvlosses[i]))
    print("------------------")

    print("\nEstimated test performance: mean and std dev")
    print("------------------")
    print("\n{}: {:.2f} (+/- {:.2f})".format(model.metrics_names[0], np.mean(cvlosses), np.std(cvlosses)))
    print("{}: {:.2f}% (+/- {:.2f}%)".format(model.metrics_names[1], np.mean(cvscores), np.std(cvscores)))

    print("\nFold Id with the highest DICE: {}".format(np.argmax(cvscores)+1))
    print(" Fold Id with the lowest loss: {}".format(np.argmin(cvscores)+1))
    
    print("\nFitting model using all training data")
    print("------------------")
    # Prepare callbacks
    callbacks = [
        ModelCheckpoint(model_path, monitor="loss", mode="min", save_best_only=True, verbose=1),
        EarlyStopping(monitor='loss', min_delta=0, patience=settings.TRN_EARLY_PATIENCE, verbose=0, mode="auto"),
        CSVLogger(csv_path, append=False),
        ]

    model = unet.build_model_softmax()
    model.compile(optimizer=opt, loss=loss, metrics=metrics)

    # Fit the model using a training set only
    start_time = time.time()
    hist = model.fit(train_imgs, train_grndtr_ext_conv,
                     epochs=settings.TRN_NUM_EPOCH,
                     batch_size=settings.TRN_BATCH_SIZE,
                     verbose=1,
                     shuffle=True,
                     callbacks=callbacks)

    print("\n\nElapsed training time: {} min".format(int((time.time() - start_time))/60))

## Plot/save the training results
Show a plot of the training history and save it to disk.

In [ ]:
if not USE_KFOLD_CV:
    plot_training_history(hist,
                          show=True,
                          save_path=settings.OUTPUT_PATH + unet.title,
                          time_stamp=True,
                          metric="dice_coef")
else:
    print("Using cross-validation, no training history saved")

## Perform pipeline test
Use the trained model on one sample in the training data set. This is just to perform pipeline testing during development.

In [ ]:
# Read training images and ground truths
train_imgs = read_images(hdf5_paths[0], settings.HDF5_KEY)
train_grndtr = read_groundtruths(hdf5_paths[1], settings.HDF5_KEY)

# Predict
predictions = model.predict(train_imgs, batch_size=settings.TRN_BATCH_SIZE, verbose=2)
print("Predictions shape: {}".format(predictions.shape))

# Convert predictions to images
predictions_imgs = convert_pred_to_img(predictions,
                                       threshold=settings.TRN_PRED_THRESHOLD,
                                       verbose=settings.VERBOSE)

# Show a single image, ground truth and segmentation map
show_image(np.squeeze(predictions_imgs[0]), 'Segmentation map')
show_image(np.squeeze(train_grndtr[0]), 'Ground truth')
show_image(np.squeeze(train_imgs[0]), 'Original image')

print("   seg map {} dtype {}".format(np.max(predictions_imgs[0]), predictions_imgs[0].dtype))
print("  gr truth {} dtype {}".format(np.max(train_grndtr[0]), train_grndtr[0].dtype))
print("  original {} dtype {}".format(np.max(train_imgs[0]), train_imgs[0].dtype))

## Training complete
The trained model is now ready to be applied to test MRI images using `DECiSION_test.ipynb`